# Bloodrager
Bloodrager barbarian critfishing build
Leshy

**Feats:**
1. Extended Reach, (free)
2. Bloodrager Dedication
3. Toughness
4. Rising Blood Magic
5.
6. Siphon Magic
7.
8. Reactive Strike
9.
10. Hematocritical

**Spells**: **cantrips** ignition or live wire, electric arc; **1st** Sure Strike; **2nd** Brine Dragon Bile; **3rd** Haste or Blazing Bolt

**Equipment:** +1 Striking Rooting Greatpick or Greatsword

In [ ]:
import xarray

import pathfinder2e_stats as pf2
from pathfinder2e_stats.tables import NPC, PC

In [ ]:
level = 10
use_hematocritical = True
use_rooting_rune = True
use_sword = False

## Attak bonus progrssion
Weapon vs. spell

In [ ]:
weapon_atk_bonus = (
    PC.level
    + PC.weapon_proficiency.sel(weapon_proficiency_col="Martial", drop=True)
    + PC.ability_bonus.sel(ability_bonus_col="4-boosts", drop=True)
    + PC.ability_bonus.sel(ability_bonus_col="Apex", drop=True)
    + PC.attack_item_bonus.sel(attack_item_bonus_col="Potency Rune", drop=True)
)

spell_atk_bonus = (
    PC.level
    + PC.spell_proficiency.sel(spell_proficiency_col="Dedication", drop=True)
    + PC.ability_bonus.sel(ability_bonus_col="2-boosts", drop=True)
)

atk_bonus = xarray.concat([weapon_atk_bonus, spell_atk_bonus], dim="kind").T
atk_bonus.coords["kind"] = ["weapon", "spell"]
atk_bonus.to_pandas()

### Let's select 3 standard targets:
- level -2 henchman, all defenses are low
- at-level monster, all defenses are moderate
- level +2 boss, all defenses are high

In [ ]:
rank = pf2.level2rank(level)

AC = xarray.concat(
    [
        NPC.AC.sel(level=level - 2, challenge="Low"),
        NPC.AC.sel(level=level + 0, challenge="Moderate"),
        NPC.AC.sel(level=level + 2, challenge="High"),
    ],
    dim="challenge",
)
saves = xarray.concat(
    [
        NPC.saving_throws.sel(level=level - 2, challenge="Low"),
        NPC.saving_throws.sel(level=level + 0, challenge="Moderate"),
        NPC.saving_throws.sel(level=level + 2, challenge="High"),
    ],
    dim="challenge",
)
defenses = xarray.concat([AC, saves], dim="kind")
defenses.coords["kind"] = ["AC", "saves"]
defenses.to_pandas()

## Build damage profiles for weapon and spells

In [ ]:
STR = PC.ability_bonus.sel(
    level=level, ability_bonus_col=["4-boosts", "Apex"], drop=True
).sum("ability_bonus_col")
weapon_specialization = PC.weapon_specialization.sel(
    level=level, weapon_specialization_col="Martial", drop=True
)
rage_weapon = PC.rage.sel(level=level, rage_col="Bloodrager weapon", drop=True)
weapon_dmg_bonus = (STR + weapon_specialization + rage_weapon).values.tolist()
rage_bleed = PC.rage.sel(
    level=level, rage_col="Bloodrager bleed", drop=True
).values.tolist()

if use_sword:
    # +1 Striking Rooting Greatsword with extended reach
    weapon = pf2.armory.swords.greatsword(2, weapon_dmg_bonus).reduce_die()
else:
    # +1 Striking Rooting Greatpick with extended reach
    weapon = pf2.armory.picks.greatpick(2, weapon_dmg_bonus).reduce_die()
    if level >= 5:
        weapon += pf2.armory.picks.critical_specialization(2)

weapon += pf2.Damage("bleed", 0, 0, rage_bleed, persistent=True)
weapon

In [ ]:
def rage_spell(
    level: int, type_: str, *, persistent: bool = False, drained: int = 2
) -> dict[pf2.DoS, list[pf2.Damage]]:
    raw = PC.rage.sel(
        level=level, rage_col=f"Bloodrager spell drained {drained}", drop=True
    ).values.tolist()
    d = pf2.Damage(type_, 0, 0, raw, persistent=persistent)
    return {
        pf2.DoS.critical_success: [d.copy(multiplier=2)],
        pf2.DoS.success: [d],
        pf2.DoS.failure: [d],
    }


ignition_melee = pf2.armory.cantrips.ignition(rank, melee=True) + rage_spell(
    level, "fire"
)
ignition_melee

In [ ]:
ignition_ranged = pf2.armory.cantrips.ignition(rank, melee=False) + rage_spell(
    level, "fire"
)
ignition_ranged

In [ ]:
live_wire = pf2.armory.cantrips.live_wire(rank) + rage_spell(level, "electricity")
live_wire

In [ ]:
electric_arc = pf2.armory.cantrips.electric_arc(rank)
electric_arc

In [ ]:
brine_dragon_bile = pf2.armory.spells.brine_dragon_bile(2) + rage_spell(
    level, "acid", persistent=True
)
brine_dragon_bile

In [ ]:
blazing_bolt_1action = pf2.armory.spells.blazing_bolt(rank=2, actions=1) + rage_spell(
    level, "fire"
)
blazing_bolt_1action

In [ ]:
blazing_bolt_3actions = pf2.armory.spells.blazing_bolt(rank=2, actions=3) + rage_spell(
    level, "fire"
)
blazing_bolt_3actions

## Attack routine
- Strike (with flank) -> Hematocritical if crit -> spell, _or_
- (if hasted) Sure Strike -> Strike (with flank) -> Hematocritical -> spell

Spell is one of:
- ignition (melee with flank)
- ignition (ranged due to reach)
- live wire
- electric arc (1-2 targets)
- blazing bolt, rank 2 (1-2-3 actions) (infinitely cycling with Syphon Magic)
- (out of round) brine dragon bile (infinitely cycling with Syphon Magic)

In [ ]:
melee_fortune = xarray.DataArray(
    [False, True],
    dims=["Sure Strike"],
    coords={"Sure Strike": [False, True]},
)

strike = pf2.damage(
    pf2.check(
        bonus=atk_bonus.sel(level=level, kind="weapon").values.tolist(),
        DC=AC - 2,
        fortune=melee_fortune,
    ),
    weapon,
)

### What are the chances of a critical hit on the weapon strike?

In [ ]:
melee_crit = strike.outcome == pf2.DoS.critical_success
melee_crit.mean("roll").to_pandas()

### The conditions of the next spell change depending on the strike and equipment
- if the strike was critical, we can use Hematocritical
- if the weapon was rooting, the target is now Clumsy 1
- if the weapon was a sword, th target is now off-guard even if not flanked

In [ ]:
hematocritical = melee_crit if use_hematocritical else xarray.zeros_like(melee_crit)
clumsy = melee_crit if use_rooting_rune else xarray.zeros_like(melee_crit).astype(int)
ranged_off_guard = 2 * melee_crit if use_sword else xarray.zeros_like(melee_crit)

### Roll damage for the spells

In [ ]:
ignition_melee_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True) - 5,
        DC=AC - 2 - clumsy,
        fortune=hematocritical,
    ),
    ignition_melee,
)

ignition_ranged_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True) - 5,
        DC=AC - clumsy - ranged_off_guard,
        fortune=hematocritical,
    ),
    ignition_ranged,
)

live_wire_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True) - 5,
        DC=AC - clumsy - ranged_off_guard,
        fortune=hematocritical,
    ),
    live_wire,
)

electric_arc_dmg = pf2.damage(
    pf2.check(
        saves - xarray.concat([clumsy, xarray.DataArray(0)], dim="target"),
        DC=spell_atk_bonus.sel(level=level, drop=True) + 10,
        misfortune=hematocritical,
    ),
    electric_arc,
)

In [ ]:
blazing_bolt_check = pf2.check(
    spell_atk_bonus.sel(level=level, drop=True) - 5,
    DC=AC
    - xarray.concat(
        [clumsy + ranged_off_guard, xarray.DataArray(0), xarray.DataArray(0)],
        dim="target",
    ),
    fortune=hematocritical,
)
blazing_bolt_1action_dmg = pf2.damage(
    blazing_bolt_check.isel(target=0, drop=True),
    blazing_bolt_1action,
)
blazing_bolt_3actions_dmg = pf2.damage(
    blazing_bolt_check,
    blazing_bolt_3actions,
)
blazing_bolt_2actions_dmg = blazing_bolt_3actions_dmg.isel(target=slice(2))

### Also show:
- A second iterative strike
- a standalon 3-actions Blazing Bolt
- an out-of-round Brine Dragon Bile

In [ ]:
strike2 = pf2.damage(
    pf2.check(
        bonus=atk_bonus.sel(level=level, kind="weapon").values.tolist() - 5,
        DC=AC - 2 - clumsy,
    ),
    weapon,
)

In [ ]:
blazing_bolt_check_noMAP = pf2.check(
    spell_atk_bonus.sel(level=level, drop=True),
    DC=AC,
    dims={"target": 3},
)
blazing_bolt_3actions_noMAP_dmg = pf2.damage(
    blazing_bolt_check_noMAP,
    blazing_bolt_3actions,
)

brine_dragon_bile_check = pf2.check(
    spell_atk_bonus.sel(level=level, drop=True),
    DC=AC,
)
brine_dragon_bile_dmg = pf2.damage(
    brine_dragon_bile_check,
    brine_dragon_bile,
)

## Mean damage for every action

In [ ]:
total_damage = xarray.concat(
    [
        strike.total_damage,
        strike2.total_damage,
        ignition_melee_dmg.total_damage,
        ignition_ranged_dmg.total_damage,
        live_wire_dmg.total_damage,
        electric_arc_dmg.total_damage.isel(target=0, drop=True),
        electric_arc_dmg.total_damage.sum("target"),
        blazing_bolt_1action_dmg.total_damage,
        blazing_bolt_2actions_dmg.total_damage.sum("target"),
        blazing_bolt_3actions_dmg.total_damage.sum("target"),
        blazing_bolt_3actions_noMAP_dmg.total_damage.sum("target"),
        brine_dragon_bile_dmg.total_damage,
    ],
    dim="spell",
    coords="minimal",
).mean("roll")
total_damage.coords["spell"] = [
    "Weapon Strike (flanked)",
    "Iterative Weapon Strike (flanked) (MAP-5)",
    "Ignition (melee, flanked) (MAP-5)",
    "Ignition (ranged) (MAP-5)",
    "Live Wire (MAP-5)",
    "Electric Arc (1 target)",
    "Electric Arc (2 targets)",
    "Blazing Bolt > (MAP-5)",
    "Blazing Bolt >> (MAP-5)",
    "Blazing Bolt >>> (MAP-5)",
    "Blazing Bolt >>> (standalone)",
    "Brine Dragon Bile (standalone)",
]
total_damage.loc[{"spell": total_damage.spell[-3:], "Sure Strike": True}] = float("nan")
total_damage.stack(col=["challenge", "Sure Strike"]).to_pandas().round(2)

### Outcome probability for the initial Strike

In [ ]:
(pf2.outcome_counts(strike).stack(col=["challenge", "Sure Strike"]).to_pandas() * 100.0)

### Outcome probability for the second Strike

In [ ]:
(
    pf2.outcome_counts(strike2).stack(col=["challenge", "Sure Strike"]).to_pandas()
    * 100.0
)

### Outcome probability for Ignition

In [ ]:
(
    pf2.outcome_counts(ignition_melee_dmg)
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    * 100.0
)

### Outcome probability for Electric Arc

In [ ]:
(
    pf2.outcome_counts(electric_arc_dmg)
    .mean("target")
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    * 100.0
)